In [19]:
CSV_file = 'data/combined_data_laliga.csv'
DB_file = 'data/Transactions_LaLiga.txt'

In [20]:
d = dict()

with open(CSV_file, 'r') as f:
    for index, line in enumerate(f.readlines()):
        line = line.strip('\n')
        values = line.split(",")
    
        if index == 0:
            values[0] = 'id'
            legend = values
            continue
    
        d[values[0]] = {}
        for i in range(1,len(values)):
            d[values[0]][legend[i]] = values[i]

In [21]:
def define_winner(match, winnerTeam, loserTeam):
    match_copy = match.copy()

    for match_key in match_copy:
        del match[match_key]

        if match_key.startswith(winnerTeam):
            new_key = match_key.replace(winnerTeam, "Winner")
            match[new_key] = match_copy[match_key]

        elif match_key.startswith(loserTeam):
            new_key = match_key.replace(loserTeam, "Loser")
            match[new_key] = match_copy[match_key]

    return line

In [22]:
keys = list(d.keys())

for key in keys:
    game = d[key]
    score = game['Score'].split('-')
    score_home, score_away = int(score[0]), int(score[1])

    if score_home > score_away:
        define_winner(game, 'Home', 'Away')

    elif score_home < score_away:
        define_winner(game, 'Away', 'Home')

    else:
        del d[key]

In [23]:
lines = list()

# Create the transactions, in the python string format
for key in d:
    line = str()
    game = d[key]

    if float(game["Winner Team Possession %"]) > 50:
        line += "Possession "

    if float(game["Winner Team Pass Success %"]) >= 85:
        line += "HighAccuracyPass "

    if float(game["Winner Team Off Target Shots"]) < float(game["Loser Team Off Target Shots"]):
        line += "LessOffTargetShots "

    if float(game["Winner Team On Target Shots"]) > float(game["Loser Team On Target Shots"]):
        line += "MoreOnTargetShots "

    if float(game["Winner Team Total Shots"]) > float(game["Loser Team Total Shots"]):
        line += "MoreShots "

    if float(game["Winner Team Aerials Won"]) > float(game["Loser Team Aerials Won"]):
        line += "MoreAerials "

    if (float(game["Loser Team Fouls"])-float(game["Winner Team Fouls"])) >= 3:
        line += "LessFouls "

    WinnerCards = float(game["Winner Team Yellow Cards"]) + float(game["Winner Team Red Cards"])
    LoserCards = float(game["Loser Team Yellow Cards"]) + float(game["Loser Team Red Cards"])
    if (LoserCards - WinnerCards) >= 2:
        line += "LessCards "

    if float(game["Winner Team Red Cards"]) < float(game["Loser Team Red Cards"]):
        line += "MorePlayers "

    # Removes the last space (' ') from the string
    line = line[:-1]

    if len(line) > 3:
        lines.append(line)

In [25]:
# Write the transaction file
with open(DB_file, 'w') as f:
    for line in lines:
        line += '\n'
        f.writelines(line)